In [22]:
pip install google-api-python-client

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install youtube_transcript_api

Defaulting to user installation because normal site-packages is not writeable
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  NOTE: The current PATH contains path(s) starting with `~`, which may not be expanded by all applications.
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from youtube_transcript_api import YouTubeTranscriptApi
import requests

/Users/joycelu/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Connect To YouTube Data API to Retrieve Top 3 Videos Related to Input

In [6]:
API_KEY = 'REDACTED_API'
CHANNEL_NAME = 'YahooFinance'

base_url = "https://www.googleapis.com/youtube/v3"

# 1. Retrieve Channel ID
def get_channel_id(channel_name):
    url = f"{base_url}/channels"
    params = {
        'part': 'id',
        'forHandle': channel_name,
        'key': API_KEY
    }
    response = requests.get(url, params=params)
    response.raise_for_status() 
    data = response.json()
    
    if 'items' in data and len(data['items']) > 0:
        return data['items'][0]['id']
    else:
        raise ValueError(f"No channel found: {channel_name}")

# 2: Get Top 3 Videos' IDs
def get_top_videos(channel_id, query, max_results=3):
    url = f"{base_url}/search"
    params = {
        'part': 'snippet',
        'channelId': channel_id,
        'q': query,
        'maxResults': max_results,
        'type': 'video',
        'key': API_KEY
    }
    response = requests.get(url, params=params)
    response.raise_for_status() 
    data = response.json()
    video_ids = [item['id']['videoId'] for item in data['items']]
    return video_ids

try:
    channel_id = get_channel_id(CHANNEL_NAME)
    company_name = 'Google'
    video_ids = get_top_videos(channel_id, company_name)
    print(video_ids)
except ValueError as e:
    print(e)
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")


['V_2_5vi71qc', 'Gbjwbq_ur_g', '-ito6YDA1N0']


### Get Transcript through Youtube Transcript API

In [3]:
def get_transcript(video_id):
        lines = YouTubeTranscriptApi.get_transcript(video_id)
        return [line['text'] for line in lines]

def preprocess_transcript(transcript):
    # Split transcript into segments of 5 lines each
    new_lst = []
    for i in range(0, len(transcript), 5):
        lines = transcript[i:i+5]
        into_one = ' '.join(lines)
        new_lst.append(into_one)
    return new_lst

In [4]:
lst = get_transcript('INGJmZaxpGE')
lst

['welcome back to the watch list',
 'semiconductors really were driving the',
 'market there was a time when the semis',
 'were up the market was up semis were',
 'down the market was down and we',
 'certainly seen some great names run up',
 "but they've since pulled back and so now",
 'the question becomes what part of this',
 'group are winners are they all winners',
 "let's bring our panel and Michael",
 'Robinson Chief technology strategist at',
 'Weiss ratings and Olivier blon shards',
 'with us research director at futurum',
 'group thank you both for being with us I',
 "mean I think there's an idea that semi",
 'certainly can lead the market um you',
 'know just saw an article on Nvidia',
 'saying that Nvidia was still a',
 'no-brainer when it comes to investing',
 'Michael what do you',
 'think well Nvidia is one of the great',
 "chip companies of all time it's just",
 "been an amazing player um you've got a",
 'kind of a bifurcated Market as it were',
 'uh a lot of the rally a

In [7]:
import requests

API_URL = "https://api-inference.huggingface.co/models/mr8488/distilroberta-finetuned-financial-news-sentiment-analysis"
headers = {"Authorization": f"Bearer {'REDACTED_API'}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": lst[0],
})

In [8]:
def get_all(lst):
    new_lst = []
    for i in range(0, len(lst), 5):
        lines = lst[i:i+5]
        into_one = ' '.join(lines)
        new_lst.append(into_one)
    print(new_lst)
    def get_sentiment(line):
        res = query({
            "inputs": line
        })
        print(res)
        return res[0][0]
    return [get_sentiment(line) for line in new_lst]

In [9]:
output[0][0]

{'label': 'neutral', 'score': 0.8874624967575073}

In [10]:
get_all(lst)

['welcome back to the watch list semiconductors really were driving the market there was a time when the semis were up the market was up semis were down the market was down and we', "certainly seen some great names run up but they've since pulled back and so now the question becomes what part of this group are winners are they all winners let's bring our panel and Michael", "Robinson Chief technology strategist at Weiss ratings and Olivier blon shards with us research director at futurum group thank you both for being with us I mean I think there's an idea that semi", 'certainly can lead the market um you know just saw an article on Nvidia saying that Nvidia was still a no-brainer when it comes to investing Michael what do you', "think well Nvidia is one of the great chip companies of all time it's just been an amazing player um you've got a kind of a bifurcated Market as it were uh a lot of the rally and Chip stocks", "that were really heavily focused on the electric vehicle Market yo

KeyboardInterrupt: 